In [1]:
!sudo apt-get install -y pciutils
!curl https://ollama.ai/install.sh | sh
!pip install ultimate-sitemap-parser langchain-community tiktoken langchain-openai langchainhub langchain langgraph langchain-text-splitters langchain-pinecone bs4 nest_asyncio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pciutils is already the newest version (1:3.7.0-6).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11868    0 11868    0     0  27831      0 --:--:-- --:--:-- --:--:-- 27793
>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
  Using cached ultimate_sitemap_parser-0.5-py2.py3-none-any.whl.metadata (4.3 kB)
  Using

**Setting up code to run ollama in the background**

In [2]:
import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

In [27]:
ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

**Pulling LLAMA3 as main LLM and nomic-embed-text for the embedding**

In [17]:
!ollama pull llama3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0

In [5]:
import pandas as pd
from usp.tree import sitemap_tree_for_homepage

def getPagesFromSitemap(fullDomain):

    listPagesRaw = []

    tree = sitemap_tree_for_homepage(fullDomain)
    for page in tree.all_pages():
        listPagesRaw.append(page.url)

    return tuple(set(listPagesRaw))


In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path
import nest_asyncio
import asyncio
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
nest_asyncio.apply()




urls = list(getPagesFromSitemap("https://docs.snowflake.com/en/"))
print("Number of pages found: ", len(urls))
print("loading pages...")
llm = ChatOllama(model="llama3", format="json", temperature=0)
total_docs=len(urls)

batch_size = 10

print(f"Total number of batches is: {total_docs/batch_size}")
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=50
  )









2024-08-07 13:51:19,968 WARNING usp.tree [2593/MainThread]: Assuming that the homepage of https://docs.snowflake.com/en/ is https://docs.snowflake.com/
2024-08-07 13:51:19,970 INFO usp.fetch_parse [2593/MainThread]: Fetching level 0 sitemap from https://docs.snowflake.com/robots.txt...
2024-08-07 13:51:19,972 INFO usp.helpers [2593/MainThread]: Fetching URL https://docs.snowflake.com/robots.txt...
2024-08-07 13:51:20,691 INFO usp.fetch_parse [2593/MainThread]: Parsing sitemap from URL https://docs.snowflake.com/robots.txt...
2024-08-07 13:51:20,693 INFO usp.fetch_parse [2593/MainThread]: Fetching level 0 sitemap from https://docs.snowflake.com/en/sitemap.xml...
2024-08-07 13:51:20,696 INFO usp.helpers [2593/MainThread]: Fetching URL https://docs.snowflake.com/en/sitemap.xml...
2024-08-07 13:51:22,995 INFO usp.fetch_parse [2593/MainThread]: Parsing sitemap from URL https://docs.snowflake.com/en/sitemap.xml...
2024-08-07 13:51:24,015 INFO usp.fetch_parse [2593/MainThread]: Fetching level

Number of pages found:  3317
loading pages...
Total number of batches is: 331.7


In [7]:
main_splits=[]
for i in range(0, total_docs, batch_size):
  # Split documents into batches

  docs = WebBaseLoader(urls[i:i+batch_size])
  docs.requests_per_second = 5
  docs.continue_on_failure=True

  docs_list =  docs.aload()
  batch_docs = docs_list

  # Create document splits
  doc_splits = text_splitter.split_documents(batch_docs)
  main_splits.extend(doc_splits)

Fetching pages: 100%|##########| 10/10 [00:02<00:00,  3.83it/s]
/usr/local/lib/python3.10/dist-packages/langchain_community/document_loaders/web_base.py:220: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  final_results.append(BeautifulSoup(result, parser, **self.bs_kwargs))
Fetching pages: 100%|##########| 7/7 [00:02<00:00,  3.22it/s]


In [28]:
prompt = PromptTemplate(
        template="""You are an expert in forming questions in the context of snowflake platform \n
        Here is the actual answer: \n\n {answer} \n\n
        I want you to come up with the question for the given answer.""",
        input_variables=["answer"],
    )

chain = prompt | llm | JsonOutputParser()

In [29]:
columns = ['Instruction', 'Input', 'Response']

#processing the whole thing is very resource intensive so lets go with 1000 splits:
main_splits=main_splits[0:1000]
df = pd.DataFrame(columns=columns)

def process_item(main_splits):
  for i, item in enumerate(main_splits):
    response = item.page_content
    output = chain.invoke({"answer": response})
    instruction = output.get('question', '')
    print(f"row {i} is set")
    yield i, response, instruction


# Using the generator to update the DataFrame
for i, response, instruction in process_item(main_splits):
    df.at[i, "Response"] = response
    df.at[i, "Instruction"] = instruction

row 0 is set
row 1 is set
row 2 is set
row 3 is set
row 4 is set
row 5 is set
row 6 is set
row 7 is set
row 8 is set
row 9 is set
row 10 is set
row 11 is set
row 12 is set
row 13 is set
row 14 is set
row 15 is set
row 16 is set
row 17 is set
row 18 is set
row 19 is set
row 20 is set
row 21 is set
row 22 is set
row 23 is set
row 24 is set
row 25 is set
row 26 is set
row 27 is set
row 28 is set
row 29 is set
row 30 is set
row 31 is set
row 32 is set
row 33 is set
row 34 is set
row 35 is set
row 36 is set
row 37 is set
row 38 is set
row 39 is set
row 40 is set
row 41 is set
row 42 is set
row 43 is set
row 44 is set
row 45 is set
row 46 is set
row 47 is set
row 48 is set
row 49 is set
row 50 is set
row 51 is set
row 52 is set
row 53 is set
row 54 is set
row 55 is set
row 56 is set
row 57 is set
row 58 is set
row 59 is set
row 60 is set
row 61 is set
row 62 is set
row 63 is set
row 64 is set
row 65 is set
row 66 is set
row 67 is set
row 68 is set
row 69 is set
row 70 is set
row 71 is set
ro

In [20]:
df

,Response,Instruction,Input
0,EXECUTE NOTEBOOK | Snowflake DocumentationDOCU...,,NaN
5,Copy\nAdd two seconds to the current date and ...,How do I add a specific number of seconds to t...,NaN
2,"Copy\nSELECT id, date1 \n FROM my_table \n ...",What is the purpose of the TO_DATE function in...,NaN
4,Copy\nGet the current month of the year by nam...,How can we get the current month of the year b...,NaN
1,Working with date and time values | Snowflake ...,How does Snowflake handle date and time values...,NaN
...,...,...,...
174,SCHEDULED_TIME | Snowflake DocumentationDOCUME...,,NaN
175,Snowflake Connector for ServiceNow® release no...,,NaN
176,New features¶\nNot applicable.\n\n\nBug fixes¶...,,NaN
177,New features¶\n\nAdded the CONFIGURE_TABLES_SC...,,NaN


In [30]:
df.to_excel("fine_tuning_input.xlsx")